# LSTM_Stock_Prediction


給定 2010 ~ 2016 紐約股市的資料：

1. `stt.csv`：State Street Corp (STT) 的資料。
2. `all.csv`：STT 以外其他公司的資料。

csv 中每一行有以下欄位：

1. `date`：日期
2. `symbol`: 公司
3. `open`：開盤價
4. `close`：收盤價
5. `low`：低點
6. `hight`：高點
7. `volume`：交易量

作業如下：

1. 使用 STT 前 15 天的資料來預測第 16 天的交易量(30%)
2. 使用所有跟 STT 同領域的公司前 15 天的資料來預測 STT 第 16 天的收盤價(30%)
3. 使用整個股市的資料前 30 天的資料來預測第 31 天的開盤價(40%)


### 以下範例程式使用STT過去15天的`open`、`close`、`low`和`volume`（without  `high`），來預測STT第16天的最高價（`high`）。

## Task:

1. Use a RNN/LSTM.
2. Plot the loss curve.
3. Report the $MSE$ between your predictions and groundtruths. (both train and valid set)

$$
MSE = \frac{1}{N} \sum_1^N (p_i - q_i)^2
$$
4. Plot the prediction vs groundtruth. (both train and valid set)


where $N$ is number of days and $p_i, q_i$ are the prediction and groudtruth on day $i$ respectively.


## Download Data and Import Libraries

In [ ]:
from pathlib import Path
from torchvision.datasets.utils import download_file_from_google_drive

csv_path_stt = Path('./stt.csv')
csv_path_all = Path('./all.csv')

if not csv_path_stt.exists():
    download_file_from_google_drive('1zY26f3Ie5c7Vt9R0wHRz9hGldOsizV4b', '.', csv_path_stt)
if not csv_path_all.exists():
    download_file_from_google_drive('1KtyafQZRtx3FOWpFEe-keoeiVTb2y7Fu', '.', csv_path_all)

Downloading...
From: https://drive.google.com/uc?id=1zY26f3Ie5c7Vt9R0wHRz9hGldOsizV4b
To: /content/stt.csv
100%|██████████| 112k/112k [00:00<00:00, 38.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KtyafQZRtx3FOWpFEe-keoeiVTb2y7Fu
To: /content/all.csv
100%|██████████| 54.5M/54.5M [00:01<00:00, 38.3MB/s]


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, Sampler

from sklearn.preprocessing import StandardScaler

In [ ]:
df_stt = pd.read_csv(csv_path_stt)      # a DataFrame object
df_stt = df_stt.sort_values(by='date')  # csv are not guarenteed to be ordered by date
df_stt.head(10)

,date,symbol,open,close,low,high,volume
0,2010-01-04,STT,43.810001,44.459999,43.549999,44.779999,3440300.0
1,2010-01-05,STT,44.520000,44.549999,44.020000,45.000000,3992500.0
2,2010-01-06,STT,44.400002,43.810001,43.650002,44.669998,4062100.0
3,2010-01-07,STT,44.349998,44.599998,43.980000,44.860001,5273100.0
4,2010-01-08,STT,44.759998,45.509998,44.389999,45.880001,6038000.0
5,2010-01-11,STT,45.889999,44.360001,44.209999,46.099998,4260000.0
6,2010-01-12,STT,43.910000,44.200001,43.910000,44.849998,3432600.0
7,2010-01-13,STT,44.500000,44.610001,43.650002,44.770000,2741000.0
8,2010-01-14,STT,44.330002,43.790001,43.189999,44.419998,4268500.0
9,2010-01-15,STT,43.520000,42.669998,42.520000,44.099998,6887100.0


### Train/Valid Split and Feature Normalization

We split first 85% of the entries to be training data, and last 15% to be validation data.
The pivot date is 2015-12-15, i.e., entries after 2015-12-15 (included) is valid data.


Note that scale of the features (`open`, `close`, `low`, `high`, `volume`) are different to each other. In practice, we normalize each features to better train our model. If we do so, the model will suffer gradient vanishment/exploding.

To normalize data, common techniques are:

1. Standard Scaler: $x_{new} = (x - mean) / std$
2. Minmax Scaler: $x_{new} = (x - x_{min}) / (x_{max} - x_{min})$

Both are implemented in `sklearn`. In this code, we use the first one.

**Be careful that when normalizing data, we can use only training data to compute the statistics. Using validation/testing data is not allowed.**


In some cases, we need to obtain original values from normalized features, we can do it using the standard scaler `std` used for normalization.

```python
raw_open = normalize_open* std.scale_[0] + std.mean_[0]
raw_close = normalized_close * std.scale_[1] + std.mean_[1]
raw_low = normalized_low * std.scale_[2] + std.mean_[2]
raw_high = normalized_low * std.scale_[3] + std.mean_[3]
raw_volume = normalized_low * std.scale_[4] + std.mean_[4]
```

In [ ]:
pivot = round(0.85 * len(df_stt))
print('pivot:', df_stt.loc[pivot, 'date'])

df_train = df_stt[:pivot].copy()    # DataFrame
df_valid = df_stt[pivot:].copy()    # DataFrame

FEATURES = ['open', 'close', 'low', 'high', 'volume']
std = StandardScaler().fit(df_train[FEATURES])
df_train[FEATURES] = std.transform(df_train[FEATURES])
df_valid[FEATURES] = std.transform(df_valid[FEATURES])

df_train.head(10)

pivot: 2015-12-15


,date,symbol,open,close,low,high,volume
0,2010-01-04,STT,-0.810652,-0.767338,-0.794788,-0.778688,-0.038990
1,2010-01-05,STT,-0.762428,-0.761233,-0.762792,-0.763806,0.254822
2,2010-01-06,STT,-0.770578,-0.811429,-0.787981,-0.786130,0.291854
3,2010-01-07,STT,-0.773975,-0.757841,-0.765515,-0.773276,0.936197
4,2010-01-08,STT,-0.746127,-0.696113,-0.737603,-0.704275,1.343181
5,2010-01-11,STT,-0.669376,-0.774121,-0.749857,-0.689393,0.397152
6,2010-01-12,STT,-0.803860,-0.784974,-0.770281,-0.773953,-0.043087
7,2010-01-13,STT,-0.763786,-0.757163,-0.787981,-0.779365,-0.411070
8,2010-01-14,STT,-0.775333,-0.812786,-0.819296,-0.803042,0.401674
9,2010-01-15,STT,-0.830349,-0.888759,-0.864908,-0.824689,1.794966


# Dataset

To predict the stock on day 16 based on day 1 ~ day 15 features, we expect each sample contains:

1. `feature`: the features of day 1 ~ 15. Each day has 4 features, thus the shape is `[15, 4]`.
2. `target`: the target we are predicting. In this case, it's the `high` of day 16, thus `target` is a shaped `[1]`.

Therefore we are performing a *sliding window*-like operation on the given dataframe.

In [ ]:
class CompanyStockData(Dataset):
    def __init__(self, dataframe, window=10):
        super().__init__()
        self.df = dataframe
        self.window = window

    def __len__(self):
        return len(self.df) - self.window

    def __getitem__(self, idx):
        window_s = idx
        window_t = idx + self.window
        feature = self.df.iloc[window_s:window_t - 1]                            # Retrieve entries [window_s, window_t - 1], DataFrame
        feature = feature[['open', 'close', 'low', 'volume']].values                    # Retrieve the wanted features. `.values` make it ndarray
        feature = torch.from_numpy(feature).float()                               # Converty ndarray to tensor
        target = self.df.iloc[window_t - 1]                                 # The entry on the next day
        target = target['high'].item()                                      # Make it python scalar
        target = torch.tensor([target]).float()                               # Make it tensor
        return feature, target


WINDOW = 16
train_set = CompanyStockData(df_train, window=WINDOW)
valid_set = CompanyStockData(df_valid, window=WINDOW)
print('#train:', len(train_set))
print('#valid:', len(valid_set))

feature, target = valid_set[100]
print(feature)
print(target)

#train: 1482
#valid: 248
tensor([[ 0.2965,  0.3621,  0.3264, -0.8257],
        [ 0.3501,  0.3126,  0.3482, -1.0176],
        [ 0.3386,  0.2888,  0.2985, -1.0392],
        [ 0.2822,  0.2346,  0.2659, -0.9518],
        [ 0.2204,  0.2671,  0.2563, -0.9916],
        [ 0.2571,  0.2658,  0.2502, -0.6379],
        [ 0.2679,  0.3885,  0.2795, -0.6177],
        [ 0.3589,  0.3356,  0.3312, -0.6860],
        [ 0.3685,  0.3838,  0.3870, -0.7537],
        [ 0.3780,  0.3716,  0.3795, -0.9759],
        [ 0.4065,  0.4598,  0.4320, -0.9083],
        [ 0.4894,  0.5364,  0.5109, -0.4874],
        [ 0.5308,  0.4815,  0.5157, -0.6934],
        [ 0.5070,  0.5127,  0.5218, -0.9226],
        [ 0.5362,  0.4944,  0.5109, -0.6456]])
tensor([0.4633])


# Model

We implements following model:



In [ ]:
class LSTM(nn.Module):
    def __init__(self, input_size=4, hidden_size=32, num_layers=1, num_classes=1):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True
        )
        self.regressor = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x, (h, c) = self.lstm(x)
        x = self.regressor(x[:, -1])        # We only takes last timestamps output of lstm
        return x

train_loader = DataLoader(
    train_set,
    shuffle=True,
    batch_size=8,
    num_workers=4,
)
valid_loader = DataLoader(
    valid_set,
    shuffle=False,
    batch_size=8,
    num_workers=4,
)

feature_b, target_b = next(iter(valid_loader))
print(feature_b.shape)  # [N, 15, 4]
print(target_b.shape)   # [N, 1]

model = LSTM(input_size=4, hidden_size=32, num_classes=1)
output_b = model(feature_b)
print(output_b.shape)   # [N, 1]

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([8, 15, 4])
torch.Size([8, 1])
torch.Size([8, 1])


Some other RNN model you can try:

In [ ]:
class BiLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BiLSTM, self).__init__()

        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers, batch_first=True, bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.fc(x[:, -1, :])
        return x


class GRU(nn.Module):
    def __init__(self, input_size, hidden_size=1, num_layers=1, num_classes=1):
        super().__init__()

        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x, _ = self.gru(x)
        x = self.fc(x[:, -1, :])
        return x

# Training

Train the model in standard way and record losses every epoch.

In [ ]:
device = 'cuda'
model = LSTM(input_size=4, hidden_size=32, num_layers=1, num_classes=1).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

history = {
    'train_loss': [],
    'valid_loss': [],
}

for epoch in tqdm(range(50)):
    # Train
    losses = []
    model.train()
    for feature_b, target_b in iter(train_loader):
        feature_b = feature_b.to(device)
        target_b = target_b.to(device)

        optimizer.zero_grad()
        predict_b = model(feature_b)
        loss = criterion(predict_b, target_b)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
    train_loss = sum(losses) / len(losses)

    # Valid
    losses = []
    model.eval()
    for feature_b, target_b in iter(valid_loader):
        feature_b = feature_b.to(device)
        target_b = target_b.to(device)
        predict_b = model(feature_b)
        loss = criterion(predict_b, target_b)
        losses.append(loss.item())
    valid_loss = sum(losses) / len(losses)

    # Logging
    history['train_loss'].append(train_loss)
    history['valid_loss'].append(valid_loss)
    print('Epoch {:03d}: {:.6f}, {:.6f}'.format(
        epoch, train_loss, valid_loss
    ))

  0%|          | 0/50 [00:00<?, ?it/s]

Epoch 000: 0.119012, 0.014963
Epoch 001: 0.008225, 0.011016
Epoch 002: 0.006444, 0.009772
Epoch 003: 0.005602, 0.009597
Epoch 004: 0.004865, 0.008767
Epoch 005: 0.004587, 0.008519
Epoch 006: 0.004389, 0.007906
Epoch 007: 0.003898, 0.007449
Epoch 008: 0.003904, 0.007213
Epoch 009: 0.003489, 0.006981
Epoch 010: 0.003209, 0.005942
Epoch 011: 0.003139, 0.005922
Epoch 012: 0.003114, 0.005568
Epoch 013: 0.002763, 0.005417
Epoch 014: 0.002792, 0.005247
Epoch 015: 0.002625, 0.005119
Epoch 016: 0.002571, 0.005264
Epoch 017: 0.002608, 0.004932
Epoch 018: 0.002451, 0.004809
Epoch 019: 0.002593, 0.005137
Epoch 020: 0.002502, 0.004861
Epoch 021: 0.002333, 0.005130
Epoch 022: 0.002433, 0.004984
Epoch 023: 0.002363, 0.004907
Epoch 024: 0.002323, 0.004527
Epoch 025: 0.002371, 0.004991
Epoch 026: 0.002288, 0.004455
Epoch 027: 0.002247, 0.004681
Epoch 028: 0.002369, 0.004356
Epoch 029: 0.002282, 0.004579
Epoch 030: 0.002303, 0.004372
Epoch 031: 0.002349, 0.004626
Epoch 032: 0.002362, 0.004576
Epoch 033:

In [ ]:
fig, ax = plt.subplots()
ax.plot(history['train_loss'], label='Train')
ax.plot(history['valid_loss'], label='Valid')
ax.set_title('Loss')
ax.legend()
plt.show()

## Evaluation



  0%|          | 0/46 [00:00<?, ?it/s]

0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
@torch.no_grad()
def evaluate(dataset):
    model.eval()
    preds = []
    trues = []
    loader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)
    for feature_b, target_b in tqdm(iter(loader)):
        predict_b = model(feature_b.to(device)).to('cpu')
        preds.extend(predict_b)
        trues.extend(target_b)
    preds = torch.stack(preds).numpy() # [N]
    trues = torch.stack(trues).numpy() # [N]
    return preds, trues


preds, trues = evaluate(train_set)
preds = preds * std.scale_[3] + std.mean_[3]
trues = trues * std.scale_[3] + std.mean_[3]
error = ((preds - trues) ** 2).mean()
fig, ax = plt.subplots()
ax.plot(trues, label='True')
ax.plot(preds, label='Pred')
ax.set_title(f'Train Error={error:.3f}')
ax.legend()
plt.show()

preds, trues = evaluate(valid_set)
preds = preds * std.scale_[3] + std.mean_[3]
trues = trues * std.scale_[3] + std.mean_[3]
error = ((preds - trues) ** 2).mean()
fig, ax = plt.subplots()
ax.plot(trues, label='True')
ax.plot(preds, label='Pred')
ax.set_title(f'Valid Error={error:.3f}')
ax.legend()
plt.show()

## Useful DataFrame methods

1. Use `df.values` to get the underlying ndarray, and use `ndarray` operations.
2. **Use Excel, by hand or other tools you want to calculate the values.**



In [ ]:
df_all = pd.read_csv(csv_path_all).sort_values('date')  # read all data of stock market

df_ual = df_all[df_all['symbol'] == 'UAL']              # select entries that has symbol 'UAL'
# df_ual = df_all[df_all['symbol'].isin(['UAL', 'WU'])] # another way, but can select multiple companies
df_ual = df_ual.reset_index(drop=True)                  # Reset the index to 0, 1, ..., 1761

print(df_ual)

df_ual = df_ual.drop(columns=['symbol', 'date'])        # drop symbol & date columns
data_ual = df_ual.values                                # get its underlying ndarray

print(data_ual.shape)
print(data_ual[:10])

            date symbol       open      close        low       high  \
0     2010-01-04    UAL  12.990000  12.800000  12.530000  13.110000   
1     2010-01-05    UAL  12.870000  13.910000  12.870000  14.100000   
2     2010-01-06    UAL  13.740000  13.270000  13.130000  13.990000   
3     2010-01-07    UAL  13.200000  13.550000  13.200000  13.950000   
4     2010-01-08    UAL  13.560000  13.330000  13.130000  13.860000   
...          ...    ...        ...        ...        ...        ...   
1757  2016-12-23    UAL  74.400002  74.790001  74.129997  74.830002   
1758  2016-12-27    UAL  74.779999  74.669998  74.500000  75.220001   
1759  2016-12-28    UAL  75.000000  73.529999  73.349998  75.180000   
1760  2016-12-29    UAL  73.529999  73.279999  72.930000  74.190002   
1761  2016-12-30    UAL  73.430000  72.879997  72.370003  73.430000   

          volume  
0      8785900.0  
1     23646300.0  
2     15398700.0  
3     10177000.0  
4      5896400.0  
...          ...  
1757   1616800

In [ ]:
df_all = pd.read_csv(csv_path_all).sort_values('date')  # read all data of stock market
df_all = df_all.drop(columns=['date'])

for symbol, group in df_all.groupby('symbol'):          # iterate different companies. group is a DataFrame that is subset of df_all
    print('symbol:', symbol, 'shape:', group.shape)
    print(group.sample(3))                              # print random 3 entries
    group = group.drop(columns=['symbol'])              # drop symbol columns
    print('mean:', group.mean(axis=0).values)           # compute mean of rest columns respectively
    print('-' * 10)

symbol: A shape: (1762, 6)
       symbol       open      close        low       high     volume
54927       A  23.183119  22.904148  22.882690  23.547926  2854800.0
537561      A  41.623749  41.816883  41.502148  41.874107   839700.0
842017      A  45.910000  46.299999  45.849998  46.320000  1937500.0
mean: [3.40652898e+01 3.40802207e+01 3.37141014e+01 3.44148416e+01
 3.92758910e+06]
----------
symbol: AAL shape: (1762, 6)
       symbol  open  close   low  high      volume
184267    AAL  6.48   6.39  6.35  6.56   8443700.0
149092    AAL  8.35   8.24  8.20  8.44   4704900.0
84880     AAL  9.49   8.96  8.84  9.53  10195300.0
mean: [2.38751078e+01 2.38737684e+01 2.34664529e+01 2.42839331e+01
 9.35403802e+06]
----------
symbol: AAP shape: (1762, 6)
       symbol        open       close         low        high     volume
649463    AAP  154.279999  153.320007  152.710007  154.889999   764100.0
385434    AAP   79.970001   78.709999   78.500000   79.970001  1462900.0
167384    AAP   59.869999 

In [ ]:
df_all = pd.read_csv(csv_path_all).sort_values('date')  # read all data of stock market
df_all = df_all.drop(columns=['date'])

groups = [] # Select those companies that has 1762 entries and symbol is started with 'Y'
for symbol, group in df_all.groupby('symbol'):          # iterate different companies. group is a DataFrame that is subset of df_all
    if len(group) == 1762 and symbol.startswith('Y'):
        groups.append(group)
df = pd.concat(groups)                                  # Concatenate those groups
df = df.reset_index(drop=True)                          # Reset the index to 0, 1, ..., 1761

print(df.shape)
print(df['symbol'].unique())                            # Companies

(3524, 6)
['YHOO' 'YUM']
